# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.

In [1]:
text = 'с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`).

In [2]:
import pandas as pd

data = pd.read_csv('preprocessed_descriptions.csv', sep=',')
data = data.dropna()
data

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


In [3]:
from nltk import word_tokenize

col = 'preprocessed_descriptions'
preproc_desc = data[col]
words = set()
for desc in preproc_desc:
    for word in word_tokenize(desc):
        if word not in words:
            words.add(word)

words = list(words)
len(words)

30823

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [4]:
import numpy as np
from nltk import edit_distance


words_edit_distance = {}
for i in range(5):
    word_pair = np.random.choice(words, size=2)
    word_pair = tuple(word_pair)
    words_edit_distance[word_pair] = edit_distance(word_pair[0], word_pair[1])

words_edit_distance

{('redbook', 'necessary'): 8,
 ('jigglers', 'palomara'): 7,
 ('beefycheesy', 'reveal'): 9,
 ('quincy', 'halva'): 6,
 ('timess', 'bakeno'): 5}

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [5]:
from heapq import nsmallest


def find_k_sim_words(word, k):
    edit_distance_list = []
    for w in words:
        edit_distance_list.append((w, edit_distance(word, w)))

    sim_words = nsmallest(k, edit_distance_list, key=lambda x: x[1])
    res = []
    for w in sim_words:
        res.append(w[0])
    return res

In [6]:
find_k_sim_words('word', 7)

['word', 'woud', 'worm', 'work', 'ward', 'wood', 'words']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами:
    * word
    * stemmed_word
    * normalized_word

Столбец `word` укажите в качестве индекса.

Для стемминга воспользуйтесь `SnowballStemmer`, для лемматизации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [7]:
from nltk import SnowballStemmer, WordNetLemmatizer

stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

table = pd.DataFrame({'word': words})
table['stemmed_word'] = table['word'].apply(lambda w: stemmer.stem(w))
table['normalized_word'] = table['word'].apply(lambda w: lemmatizer.lemmatize(w))
table = table.set_index('word')
table

,stemmed_word,normalized_word
word,,
schoolchildren,schoolchildren,schoolchildren
panswrap,panswrap,panswrap
chair,chair,chair
colt,colt,colt
daves,dave,daves
...,...,...
quinoas,quinoa,quinoas
faye,fay,faye
simmered,simmer,simmered


In [8]:
# совпадает результатов стемминга и лемматизации
(table['stemmed_word'] == table['normalized_word']).sum()

17424

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [9]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
total_word_list = []
clear_word_list = []
for desc in preproc_desc:
    for word in word_tokenize(desc):
        total_word_list.append(word)
        if word not in stop_words:
            clear_word_list.append(word)

print('Всего:')
print(len(total_word_list))
print('Без стоп-слов:')
print(len(clear_word_list))

Всего:
1071865
Без стоп-слов:
581919


In [10]:
print('Доля:')
print((len(total_word_list) - len(clear_word_list)) / len(total_word_list))

Доля:
0.4570967425935169


In [11]:
from collections import Counter

print('Топ-10 по частоте (всех)')
print(Counter(total_word_list).most_common(10))
print('Топ-10 по частоте (без стоп-слов:)')
print(Counter(clear_word_list).most_common(10))

Топ-10 по частоте (всех)
[('the', 40210), ('a', 34994), ('and', 30279), ('this', 27048), ('i', 25111), ('to', 23499), ('is', 20290), ('it', 19863), ('of', 18372), ('for', 15988)]
Топ-10 по частоте (без стоп-слов:)
[('recipe', 14957), ('make', 6353), ('time', 5180), ('use', 4635), ('great', 4453), ('like', 4175), ('easy', 4175), ('one', 3886), ('good', 3820), ('made', 3814)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [12]:
data_rand_5 = data.sample(5)
data_rand_5

,name,preprocessed_descriptions
10224,easy southern oven fried chicken,quick and easy from newspaper one of most requ...
5640,chicken apple grape salad,very refreshing on a hot summer night
26748,sweet potato souffle by bessinger s bar b que,as served on the buffet at the one in charlest...
3612,braised onions a la julia child,these onions are included in the recipe for bo...
5351,chef lyles homemade spaghetti sauce,this is my grandmothers recipe with a chefs im...


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vectorizer = TfidfVectorizer()
tfid_vectorizer.fit(data_rand_5[col])
data_rand_5['tfid'] = data_rand_5[col].apply(lambda x: tfid_vectorizer.transform([x]).toarray())
data_rand_5

,name,preprocessed_descriptions,tfid
10224,easy southern oven fried chicken,quick and easy from newspaper one of most requ...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.1938635048628883,..."
5640,chicken apple grape salad,very refreshing on a hot summer night,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
26748,sweet potato souffle by bessinger s bar b que,as served on the buffet at the one in charlest...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3445741..."
3612,braised onions a la julia child,these onions are included in the recipe for bo...,"[[0.0, 0.09489598848871032, 0.0, 0.0, 0.0, 0.0..."
5351,chef lyles homemade spaghetti sauce,this is my grandmothers recipe with a chefs im...,"[[0.06888887131706582, 0.0, 0.0688888713170658..."


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [14]:
from scipy.spatial.distance import cosine

cosine_array = np.zeros((len(data_rand_5['tfid'].values), len(data_rand_5['tfid'].values)))
for i, u in enumerate(data_rand_5['tfid']):
    for j, v in enumerate(data_rand_5['tfid']):
        cosine_array[i, j] = cosine(u, v)

cosine_data = pd.DataFrame(cosine_array, index=data_rand_5['name'], columns=data_rand_5['name'])
cosine_data

name,easy southern oven fried chicken,chicken apple grape salad,sweet potato souffle by bessinger s bar b que,braised onions a la julia child,chef lyles homemade spaghetti sauce
name,,,,,
easy southern oven fried chicken,0.000000,0.976136,0.927306,0.933459,0.832765
chicken apple grape salad,0.976136,0.000000,0.965779,0.990576,0.986317
sweet potato souffle by bessinger s bar b que,0.927306,0.965779,0.000000,0.818015,0.818876
braised onions a la julia child,0.933459,0.990576,0.818015,0.000000,0.711149
chef lyles homemade spaghetti sauce,0.832765,0.986317,0.818876,0.711149,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [15]:
print(cosine([3, 2, 4], [3, 2, 4]))
print(cosine([3, 2, 4], [3, 2, 2]))

0
0.0542075668668055


In [16]:
# Если косинусное расстояние равняется 0, то описания идентичны
# Если косинусное расстояние в промежутке (0, 1), то есть схожесть между описаниями
# Если косинусное расстояние равняется 1, то тексты совсем не похожи
